<a href="https://colab.research.google.com/github/noviantisafitri/Jaringan-Jalan-Kaltim/blob/main/Jalan_Arteri_dan_Kolektor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install library yang dibutuhkan (jika belum)
!pip install osmnx geopandas pyproj fiona shapely rtree

In [ ]:
# 2. Import library
import osmnx as ox
import geopandas as gpd
import shutil
from google.colab import files

In [ ]:
# 3. Tentukan kota
kota = "Bontang, Kalimantan Timur, Indonesia"

# 4. Ambil jaringan jalan kendaraan bermotor
graph = ox.graph_from_place(kota, network_type='drive')

# 5. Konversi ke GeoDataFrame (hanya edges / ruas jalan)
gdf_jalan = ox.graph_to_gdfs(graph, nodes=False, edges=True)

# 6. Filter: hanya jalan yang punya nama
gdf_jalan = gdf_jalan[~gdf_jalan['name'].isna()]

# 7. Definisikan kategori jalan
arteri = ['motorway', 'trunk', 'primary']
kolektor = ['secondary', 'tertiary']
kategori_terpilih = arteri + kolektor

# 8. Filter hanya jalan arteri dan kolektor
gdf_jalan = gdf_jalan[gdf_jalan['highway'].isin(kategori_terpilih)]

In [ ]:
# 9. Tambahkan kolom kategori
def kategorikan_jalan(jenis):
    if jenis in arteri:
        return "Arteri"
    elif jenis in kolektor:
        return "Kolektor"
    else:
        return "Lainnya"

gdf_jalan['kategori'] = gdf_jalan['highway'].apply(kategorikan_jalan)

In [ ]:
# 10. Reset index agar rapi
gdf_jalan = gdf_jalan.reset_index(drop=True)

# 11. Simpan ke dalam folder sebagai shapefile
output_folder = "/content/jalan_arteri_kolektor_bontang"
gdf_jalan.to_file(output_folder, driver="ESRI Shapefile")

In [ ]:
# 12. Kompres hasil shapefile menjadi ZIP
zip_path = shutil.make_archive("jalan_arteri_kolektor_bontang", 'zip', output_folder)

In [ ]:
# 13. Unduh file ZIP ke lokal
files.download(zip_path)